In [1]:
using Pkg; Pkg.activate("."); Pkg.instantiate();

using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated
using CuArrays

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
┌ Warning: Error requiring CuArrays from Flux:
│ LoadError: UndefVarError: cudnn_available not defined
│ Stacktrace:
│  [1] getproperty(::Module, ::Symbol) at ./sysimg.jl:13
│  [2] top-level scope at none:0
│  [3] include at ./boot.jl:317 [inlined]
│  [4] include_relative(::Module, ::String) at ./loading.jl:1038
│  [5] include at ./sysimg.jl:29 [inlined]
│  [6] include(::String) at /home/dhairyagandhi96/.julia/packages/Flux/kJVo6/src/Flux.jl:1
│  [7] top-level scope at none:0
│  [8] eval at ./boot.jl:319 [inlined]
│  [9] eval at /home/dhairyagandhi96/.julia/packages/Flux/kJVo6/src/Flux.jl:1 [inlined]
│  [10] (::getfield(Flux, Symbol("##97#101")))() at /home/dhairyagandhi96/.julia/packages/Requires/9Jse/src/require.jl:67
│  [11] err(::getfield(Flux, Symbol("##97#101")), ::Module, ::String) at /home/dhairyagandhi96/.julia/packages/Requires/9Jse/src/require.jl:38
│  [1

Classify MNIST digits with a simple multi-layer-perceptron

In [2]:
imgs = MNIST.images()

60000-element Array{Array{ColorTypes.Gray{FixedPointNumbers.Normed{UInt8,8}},2},1}:
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8

Stack images into one large batch

In [3]:
X = hcat(float.(reshape.(imgs, :))...) |> gpu

labels = MNIST.labels()

60000-element Array{Int64,1}:
 5
 0
 4
 1
 9
 2
 1
 3
 1
 4
 ⋮
 2
 9
 5
 1
 8
 3
 5
 6
 8

One-hot-encode the labels

In [4]:
Y = onehotbatch(labels, 0:9) |> gpu

m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax) |> gpu

loss(x, y) = crossentropy(m(x), y)

accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

dataset = repeated((X, Y), 200)
evalcb = () -> @show(loss(X, Y))
opt = ADAM(params(m))

Flux.train!(loss, dataset, opt, cb = throttle(evalcb, 10))

accuracy(X, Y)

loss(X, Y) = 2.267993f0 (tracked)


0.9205166666666666

Test set accuracy

In [5]:
tX = hcat(float.(reshape.(MNIST.images(:test), :))...) |> gpu
tY = onehotbatch(MNIST.labels(:test), 0:9) |> gpu

accuracy(tX, tY)

0.9221